# Calibration and evaluation of a rainfall-runoff model

Imagine that we want to simulate the natural inflows into a water reservoir, knowing the amount of rainfall that has fallen in the reservoir’s catchment area. For that purpose, we can use a rainfall-runoff model. A rainfall-runoff model is a mathematical model describing the rainfall–runoff processes that occur in a watershed. The model consists of a set of equations, which describe the various processes of soil infiltration, surface and subsurface runoff, etc., as a function of various parameters, which describe the hydrological characteristics of the watershed, ultimately enabling the estimation of the flow at selected river sections.

To tailor a generic rainfall-runoff model to a particular catchment, model calibration is required. Model calibration is the process of adjusting the model parameters to obtain a representation of the system under study that satisfies pre-agreed criteria. Normally, calibration of a rainfall-runoff model aims to improve the fit of the simulated flows to observed flows and involves running the model many times under different combinations of the parameter values, until a combination is found that minimise the differences between simulated and observed flows. 
 

## The HBV rainfall-runoff model: general structure
In this example, we will use the HBV rainfall-runoff model [(Bergström, 1992)](https://www.smhi.se/en/publications/the-hbv-model-its-structure-and-applications-1.83591). The HBV model is a lumped hydrological model, meaning that all the processes included in the model are spatially aggregated into “conceptual” representations at the catchment scale.   In brief, the structure, forcing inputs, parameters and output of the model are the following.

#### Structure

The model consists of four main modules/subroutines: 
1. **SM module**: for soil moisture (***SM***), actual evapotranspiration (***EA***) and recharge estimation (***R***)
2. **UZ module**: for upper zone runoff generation (***Q0*** = surface runoff + interflow) and percolation (***PERC*** = water flux from upper to lower zone) 
3. **LZ module**: for lower zone runoff generation (***Q1*** = baseflow)
4. **Routing module**: for runoff routing.

<left><img src="../../Toolbox/Inflow_simulation/HBV model structure.png" width = "800px"><left>

#### Forcing inputs

The forcing inputs of the model simulation are time series of observed precipitation (***P***) and estimated potential evapotranspiration (***E***) – these are the spatial averages of precipitation and evapotranspiration across the watershed area. Usually these time series are given at daily resolution, and this will be the case in our example too, but it is possible to use shorter time steps.

#### Model parameters:
In order to tailor the general model equations to the particular watershed under study, we need to specify the watershed surface area, and a number of other parameters that characterise the climate, geology, soil properties, etc. of that place. These parameters are: 


1. ***SSM0***    = initial soil moisture [mm]
2. ***SUZ0***    = initial Upper Zone storage [mm]
3. ***SLZ0***    = initial Lower Zone storage [mm]
4. ***BETA***    = Exponential parameter in soil routine [-]
5. ***LP***      = Limit for potential evapotranspiration [-]
6. ***FC***      = Maximum soil moisture content [mm] 
7. ***PERC***    = Maximum flux from Upper to Lower Zone [mm/day]
8. ***K0***      = Near surface flow coefficient
9. ***K1***      = Recession coefficient for the Upper Zone (ratio) [1/day]
10. ***K2***     = Recession coefficient for the Lower Zone (ratio) [1/day]
11. ***UZL***    = Near surface flow threshold [mm]
12. ***MAXBAS*** = Transfer function parameter [day]

#### Model outputs
For a given selection of the model parameters and forcing input time series, the model simulation returns time series of the following state and flux variables: 

1. ***EA***    = Actual Evapotranspiration [mm/day]
2. ***SM***    = Soil Moisture [mm]
3. ***R***     = Recharge (water flow from Soil to Upper Zone) [mm/day]
4. ***UZ***    = Upper Zone water content [mm]
5. ***LZ***    = Lower Zone water content [mm]
6. ***RL***    = Recharge to the Lower Zone [mm]
7. ***Q0***    = Water flow from Upper Zone [ML/day]
8. ***Q1***    = Water flow from Lower Zone [ML/day]
9. ***Qsim*** = Total water flow [ML/day]

To run the model we need to import some necessary libraries. **Only if iRONs is run locally**: since three required libraries, [Bqplot](https://bqplot.readthedocs.io/), [Netcdf4](https://pypi.org/project/netCDF4/) and [Platypus](https://platypus.readthedocs.io/en/latest/#), are not available on Anaconda by default, you must have installed them first. Help on how to install such libraries is given here: [How to install libraries](../0%20-%20Tutorials/0.b%20-%20How%20to%20install%20libraries.ipynb). If iRONs is run on the cloud, e.g. on Binder or Microsoft Azure Notebooks, we do not need to install the libraries to import them. 

Once the libraries are installed or in the case that we are running iRONs on the cloud, we can import them with the following code:

In [1]:
# Libraries for visualization and interactivity
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import ipywidgets as widgets
from IPython.display import display
# Library for scientific computing
import numpy as np
# Library for manipulating dates and times
from datetime import datetime, timedelta
# # Library for general purposes
import sys
warnings.filterwarnings('ignore') # to ignore warning messages

We also need to import several tools from the iRONs toolbox:

In [2]:
sys.path.append('../../Toolbox/')
from Inflow_simulation.HBV_sim import HBV_sim # HBV model
from Inflow_simulation.HBV_calibration import HBV_calibration # HBV model calibration

# Application to the Wimbleball reservoir's catchment

The catchment is, located in the south-west of England, it has a drainage area of 28.8 km2, and collects water from the river Haddeo and drains into the Wimbleball reservoir.  

## Loading and visualizing data

### Area
Let’s first define the extent of the watershed surface area.

In [3]:
area = 28.8 # km2

### Climate data 
We call a sub-routine to load daily historical climate data (evapotranspiration, precipitation and temperature) of our study area for the year 2000.

In [4]:
from Modules.Historical_data import Climate_data, Flow_data # To load historical climate and streamflow data
cal_year = 2000
clim_date, E, P, T = Climate_data(cal_year)

Plotting the precipitation data

In [5]:
# Let's create a scale for the x attribute, and a scale for the y attribute
x_sc_1 = DateScale()
y_sc_1 = LinearScale()

x_ax_1 = Axis(label='date', scale=x_sc_1)
y_ax_1 = Axis(label='mm/day', scale=y_sc_1, orientation='vertical')

fig_1 = plt.figure(title = 'daily precipitation', axes=[x_ax_1, y_ax_1], scales={'x': x_sc_1, 'y': y_sc_1},
                   layout={'min_width': '1000px', 'max_height': '300px'})
precip_bars = plt.bar(clim_date,P,colors=['blue'],stroke = 'lightgray')
fig_1

Figure(axes=[Axis(label='date', scale=DateScale()), Axis(label='mm/day', orientation='vertical', scale=LinearS…

### Observed flow
We call a sub-routine to load daily historical flow data of our watershed area for the year 2000 (this data will be used to compare against the model predictions).

In [6]:
Q_obs_date, Q_obs = Flow_data(cal_year)

Plotting the observed flow data

In [7]:
x_sc_2 = DateScale()
y_sc_2 = LinearScale(max=1200)

x_ax_2 = Axis(label='date', scale=x_sc_2)
y_ax_2 = Axis(label='ML/day', scale=y_sc_2, orientation='vertical')

fig_2 = plt.figure(title = 'observed daily water flow', axes=[x_ax_2, y_ax_2],scales={'x': x_sc_2, 'y': y_sc_2},
                   layout={'min_width': '1000px', 'max_height': '300px'})
obs_flow = plt.plot(Q_obs_date,Q_obs,colors=['black'])
fig_2

Figure(axes=[Axis(label='date', scale=DateScale()), Axis(label='ML/day', orientation='vertical', scale=LinearS…

# Manual model calibration

First we will try to calibrate the model manually, that is, changing the parameter values one at the time and looking at the effects induced in the model predictions by means of an interactive plot of the simulated hydrograph. The objective is to obtain a good fit of the simulated hydrograph to the observed one.

To measure the goodness-of-fit between the simulated and the observed flow we will use the root mean square error (RMSE). RMSE is the standard deviation of the prediction errors, i.e. the difference between the simulated (***s(t)***) and the observed (***o(t)***) hydrograph.
$$RMSE = \sqrt{\frac{\sum_{t=0}^{T} (s(t)-o(t))^{2}}{T}}$$

First, let’s execute the code below to define the sliders that will later appear in the interactive hydrograph: 

In [8]:
# Interactive sliders definition
def update_sim_hyd(P,E,param,Case,ini):
    Q_sim,[SM,UZ,LZ],[EA,R,RL,Q0,Q1] = HBV_sim(P,E,param,Case,ini,area)
    RMSE = np.sqrt(((Q_sim - Q_obs) ** 2).mean())
    return Q_sim,RMSE

def params_changed(change):
    y_vals = update_sim_hyd(P,E,[BETA.value, LP.value, FC.value, PERC.value, K0.value, K1.value, K2.value, UZL.value, MAXBAS.value],1,[SSM0.value,SUZ0.value,SLZ0.value])[0]
    RMSE = update_sim_hyd(P,E,[BETA.value, LP.value, FC.value, PERC.value, K0.value, K1.value, K2.value, UZL.value, MAXBAS.value],1,[SSM0.value,SUZ0.value,SLZ0.value])[1]
    sim_hyd.y = y_vals
    fig_3.title = 'RMSE = ' +str("%.2f" % RMSE)

SSM0 = widgets.FloatSlider(min = 0, max = 400, step=10, value = 200, description = 'Initial soil moisture ($mm$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
SSM0.observe(params_changed,'value')
SUZ0 = widgets.FloatSlider(min = 0, max = 100, step=.5, value = 50, description = 'Initial water content of UZ ($mm$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
SUZ0.observe(params_changed,'value')
SLZ0 = widgets.FloatSlider(min = 0, max = 100, step=.5, value = 50, description = 'Initial water content of LZ ($mm$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
SLZ0.observe(params_changed,'value')
BETA = widgets.FloatSlider(min = 0, max = 7, value = 3.5, description = 'Exponential parameter in soil routine (-)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
BETA.observe(params_changed,'value')
LP = widgets.FloatSlider(min = 0.3, max = 1, step=0.05, value = 0.65, description = 'Limit for potential evapotranspiration (-)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
LP.observe(params_changed,'value')
FC = widgets.FloatSlider(min = 1, max = 2000, value = 1000, description = 'Maximum soil moisture content ($mm$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
FC.observe(params_changed,'value')
PERC = widgets.FloatSlider(min = 0, max = 100, value = 50, description = 'Maximum flow from UZ to LZ ($mm$ $day^{-1}$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
PERC.observe(params_changed,'value')
K0 = widgets.FloatSlider(min = 0, max = 2, step=0.05, value = 1, description = 'Near surface flow coefficient (-)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
K0.observe(params_changed,'value')
K1 = widgets.FloatSlider(min = 0, max = 1, value = 0.5, description = 'Recession coefficient for UZ ($day^{-1}$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
K1.observe(params_changed,'value')
K2 = widgets.FloatSlider(min = 0, max = 0.1, step=0.005, value = 0.05, description = 'Recession coefficient for LZ ($day^{-1}$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
K2.observe(params_changed,'value')
UZL = widgets.FloatSlider(min = 0, max = 100, value = 50, description = 'Near surface flow threshold ($mm$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
UZL.observe(params_changed,'value')
MAXBAS = widgets.FloatSlider(min = 1, max = 6, step=0.5, value = 3.5, description = 'Transfer function parameter ($day$)',style = {'description_width': '300px'} ,layout={'width': '1000px'})
MAXBAS.observe(params_changed,'value')

Now, let’s play with the interactive hydrograph! To start try to visually fit the simulated hydrograph to the observed one in January by playing only with the parameters defining the initial conditions and the **Maximum soil moisture content**:

In [9]:
x_ax_3 = Axis(label='date', scale=x_sc_2)
y_ax_3 = Axis(label='ML/day', scale=y_sc_2, orientation='vertical')

ini = [SSM0.value,SUZ0.value,SLZ0.value]
param = [BETA.value, LP.value, FC.value, PERC.value, K0.value, K1.value, K2.value, UZL.value, MAXBAS.value]

fig_3 = plt.figure(title = 'RMSE = ' +str("%.2f ML/day" % update_sim_hyd(P,E,param,1,ini)[1]), axes=[x_ax_3, y_ax_3],scales={'x': x_sc_2, 'y': y_sc_2}, layout={'min_width': '1000px', 'max_height': '300px'}, animation_duration=1000)
obs_hyd = plt.plot(Q_obs_date,Q_obs,colors=['black'])
sim_hyd = plt.plot(Q_obs_date,update_sim_hyd(P,E,param,1,ini)[0])
sim_hyd.observe(params_changed, ['x', 'y'])

widgets.VBox([fig_3,SSM0,SUZ0,SLZ0,BETA, LP, FC, PERC, K0, K1, K2, UZL, MAXBAS])

Then play with the value of the other parameters and try to get close or even lower than **RMSE = 90 ML/day**.
We can see the complexity of manual calibration: it is difficult to find the combination of parameters that optimally fits the simulated hydrograph to the observed one.

# Automatic model calibration
<left><img src="../../util/images/AutomaticOptimization2.gif" width = "600px"><left>
In order to facilitate the search for an optimal parameter combination we can apply an automatic optimization algorithm. In the cell below, the function HBV_calibration calls an optimization algorithm (the genetic algorithm NSGAII [(Deb et al, 2002)](https://ieeexplore.ieee.org/document/996017) from the [Platypus library](https://platypus.readthedocs.io/en/latest/#)) runs the model 1000 times to find among different parameter combinations the set of input parameters (***ini_all*** and ***param_all***) that best match the simulated flow with the observed one, i.e. the objective of the algorithm is minimize the RMSE value. **Comment:** Please be aware  that it may take a few seconds to run the cell below (while running you will see **In [*]** at the upper left-hand of the cell and when the computation has finished you will see a number between the brackets)

In [18]:
cal_objective = 'all' # the objective is to minimize RMSE considering ALL the hydrograph  
iterations = 1000 # number of iterations
results_all,solution_all, RMSE_all = HBV_calibration(P,E,1,area, Q_obs, cal_objective,iterations)
ini_all = solution_all[0][0:3] # Initial conditions
param_all = solution_all[0][3:13] # Model parameters
Q_sim_all,[SM,UZ,LZ],[EA,R,RL,Q0,Q1] = HBV_sim(P,E,param_all,1,ini_all,area) # Simulation using the optimal set of parameters

#### Plot the automatically calibrated hydrograph vs the observed 

In [19]:
Case = 1
Q_sim_all,[SM,UZ,LZ],[EA,R,RL,Q0,Q1] = HBV_sim(P,E,param_all,Case,ini_all,area)
x_ax_4 = Axis(label='date', scale=x_sc_2)
y_ax_4 = Axis(label='ML/day', scale=y_sc_2, orientation='vertical')
         
fig_4 = plt.figure(title = 'RMSE = '+str("%.2f ML/day" % RMSE_all[0]), axes=[x_ax_4, y_ax_4],scales={'x': x_sc_2, 'y': y_sc_2},
                   layout={'min_width': '1000px', 'max_height': '300px'}, animation_duration=1000)
obs_hyd = plt.plot(Q_obs_date,Q_obs,colors=['black'])
sim_hyd_all = plt.plot(Q_obs_date,Q_sim_all)
fig_4

Figure(animation_duration=1000, axes=[Axis(label='date', scale=DateScale()), Axis(label='ML/day', orientation=…

***Comment:*** overall, the hydrograph of the automatically calibrated model seems to fit the observations quite well. However, if we look at low flow periods in particular, for example August and September 2000, we see that the model predictions tend to systematically overestimate the flows.  

### Objective: improve the prediction of the low flows
Imagine that our reservoir operation is very sensitive to low flows, for example because it is in low flow periods that exceptional supply management measures must be put in place. Then we would like our rainfall-runoff model to be more accurate in the prediction of the low flows, rather than the high flows. To achieve this, we can re-define the objective function as the RMSE of only the part of the hydrograph below the 50% percentile. Let’s visualise this part of the hydrograph:

In [20]:
x_ax_5 = Axis(label='date', scale=x_sc_2)
y_ax_5 = Axis(label='ML/day', scale=y_sc_2, orientation='vertical')
         
fig_5 = plt.figure(title = 'Low flows: observed daily water flow < 50th percentile', axes=[x_ax_5, y_ax_5],scales={'x': x_sc_2, 'y': y_sc_2},layout={'min_width': '1000px', 'max_height': '300px'}, animation_duration=1000)
obs_hyd_lt50 = plt.plot(x=Q_obs_date,y=Q_obs,colors=['black'])
lt50 = plt.plot(x=Q_obs_date,y=np.minimum(Q_obs/Q_obs*np.percentile(Q_obs, 50),Q_obs),opacities = [0], fill = 'bottom',fill_opacities = [0.5])
p50  = plt.plot(x=Q_obs_date,y=Q_obs/Q_obs*np.percentile(Q_obs, 50), line_style = 'dashed')
fig_5

Figure(animation_duration=1000, axes=[Axis(label='date', scale=DateScale()), Axis(label='ML/day', orientation=…

… and re-run the automatic calibration algorithm by using this new definition of the RMSE:

In [13]:
cal_objective = 'low' # the objective is to minimize RMSE considering only low flows 
results_low,solution_low, RMSE_low = HBV_calibration(P,E,Case,area, Q_obs, cal_objective,iterations)
ini_low = solution_low[0][0:3] # Initial conditions
param_low = solution_low[0][3:13] # Model parameters
Q_sim_low,[SM,UZ,LZ],[EA,R,RL,Q0,Q1] = HBV_sim(P,E,param_low,Case,ini_low,area)

#### Plot the automatically hydrograph vs the observed 

In [14]:
x_ax_6 = Axis(label='date', scale=x_sc_2)
y_ax_6 = Axis(label='ML/day', scale=y_sc_2, orientation='vertical')
         
fig_6 = plt.figure(title = 'RMSE = '+str("%.2f ML/day" % RMSE_low[0]), axes=[x_ax_6, y_ax_6],scales={'x': x_sc_2, 'y': y_sc_2},layout={'min_width': '1000px', 'max_height': '300px'}, animation_duration=1000)
obs_hyd = plt.plot(Q_obs_date,Q_obs,colors=['black'])
sim_hyd_low = plt.plot(Q_obs_date,Q_sim_low)
fig_6

Figure(animation_duration=1000, axes=[Axis(label='date', scale=DateScale()), Axis(label='ML/day', orientation=…

***Comment:*** now the simulated hydrograph over the low flow periods August-September 2000 is much closer to the observations, but this comes at the expense of completely misrepresenting high flows! 

### Trading-off between conflicting objectives 
As we have seen in the previous example, the goodness-of-fit (as measured by the RMSE) between simulated and observed hydrograph is quite poor when the calibration aims to improve the low flow predictions only, because improving on low flows leads to much poorer predictions of all other flows. So there is a tradeoff between the two objective functions, and we may want to investigate this tradeoff and look for a parameter set that produces a ‘sensible compromise’. We can do this by using a multi-objective optimisation algorithm [(Yapo et al, 1998)](https://www.sciencedirect.com/science/article/pii/S0022169497001078), which will find a set of parameter combinations that realise different ‘optimal’ compromises between fitting high and low flows (also called Pareto-optimal solutions) [Learn more about the Pareto optimality](https://www.youtube.com/watch?v=cT3DcuZnsGs)

In [15]:
cal_objective = 'double' # two objectives (RMSE of low and high flows)
population_size = 100 # number of Pareto-optimal solutions
results_double_low,results_double_high,solution_double, RMSE_double = HBV_calibration(
    P,E,Case,area, Q_obs, cal_objective,iterations,population_size)

#### Plot the interactive Pareto front
Now select the set of parameters that produces a more "sensible compromise" between the objectives by clicking on the Pareto front points.

In [16]:
# Interactive Pareto front code (Calibration)
def update_sol_hyd(i):
    ini_double = solution_double[i][0:3]
    param_double = solution_double[i][3:]
    Q_sim_double,[SM,UZ,LZ],[EA,R,RL,Q0,Q1] = HBV_sim(P,E,param_double,Case,ini_double,area)
    RMSE = RMSE_double[i]
    fig_7.title = 'RMSE = '+str("%.2f ML/day" % RMSE)
    return Q_sim_double, RMSE, i

def solution_selected(change):
    if pareto_front.selected == None:
        pareto_front.selected = [0]
    y_vals = update_sol_hyd(pareto_front.selected[0])[0]
    sim_hyd_double.y = y_vals

x_sc_pf = LinearScale()
y_sc_pf = LinearScale()

x_ax_pf = Axis(label='RMSE Low flows', scale=x_sc_pf)
y_ax_pf = Axis(label='RMSE High flows', scale=y_sc_pf, orientation='vertical')
  
fig_pf = plt.figure(title = 'Interactive Pareto front (Calibration)', axes=[x_ax_pf, y_ax_pf],
                    layout={'width': '800px', 'height': '600px'}, animation_duration=1000)
pareto_front = plt.scatter(results_double_low[:],results_double_high[:],scales={'x': x_sc_pf, 'y': y_sc_pf},
                           colors=['deepskyblue'], interactions={'hover':'tooltip','click': 'select'})
pareto_front.unselected_style={'opacity': 0.4}
pareto_front.selected_style={'fill': 'red', 'stroke': 'yellow', 'width': '1125px', 'height': '125px'}
def_tt = Tooltip(fields=['x', 'y'],labels=['RMSE (High flows)', 'RMSE (Low flows)'], formats=['.1f', '.1f'])
pareto_front.tooltip=def_tt
pareto_front.selected = [0]

pareto_front.observe(solution_selected,'selected')    

x_sc_7 = DateScale()
y_sc_7 = LinearScale(max=1000)
    
x_ax_7 = Axis(label='date', scale=x_sc_7)
y_ax_7 = Axis(label='ML/day', scale=y_sc_7, orientation='vertical')

fig_7 = plt.figure(axes=[x_ax_7, y_ax_7], layout={'min_width': '1000px', 'max_height': '300px'}, animation_duration=1000,
                   scales={'x': x_sc_7, 'y': y_sc_7})
obs_hyd = plt.plot(Q_obs_date,Q_obs,colors=['black'])
sim_hyd_double = plt.plot(Q_obs_date,update_sol_hyd(pareto_front.selected[0])[0])
sim_hyd_double.observe(solution_selected, ['x', 'y'])
plt.VBox([fig_pf,fig_7])

# Evaluation of the calibrated model against new data
The calibration results that we have looked at so far, and in particular the values of the RMSE over the high and low flows, were based on the model simulations for the year 2000, that is, the same year that was used to calibrate the model in the first place. But how would the model perform when presented with new data, for instance those of the following year? To answer this question, we can run model simulations using the previously selected calibration (represented with a cross) against the forcing data of 2001, and calculate the RMSE values for this new year.

In [21]:
# Interactive Pareto front code (Validation)
val_year = cal_year + 1 # following year
results_double_low_val = np.zeros(population_size); results_double_high_val = np.zeros(population_size)
RMSE_double_val = np.zeros(population_size)
clim_date_val, E_val, P_val, T_val = Climate_data(val_year)
Q_obs_date_val, Q_obs_val = Flow_data(val_year)
P_val = P_val*2
Q_obs_val = Q_obs_val*2

for i in range(population_size):

    Q_sim_double_val,[SM,UZ,LZ],[EA,R,RL,Q0,Q1] = HBV_sim(P_val,E_val,solution_double[i][3:],
                                                          Case,solution_double[i][0:3],area)
    
    high_flow_indexes = [Q_obs_val > np.percentile(Q_obs_val,50)]
    Q_obs_high_val = Q_obs_val[high_flow_indexes]
    Q_sim_double_high_val = Q_sim_double_val[high_flow_indexes]

    low_flow_indexes = [Q_obs_val < np.percentile(Q_obs_val,50)]
    Q_obs_low_val = Q_obs_val[low_flow_indexes]
    Q_sim_double_low_val = Q_sim_double_val[low_flow_indexes]
    
    results_double_low_val[i]       = np.sqrt(((Q_sim_double_low_val - Q_obs_low_val) ** 2).mean())
    results_double_high_val[i]       = np.sqrt(((Q_sim_double_high_val - Q_obs_high_val) ** 2).mean())
    RMSE_double_val[i]        = np.sqrt(((Q_sim_double_val - Q_obs_val) ** 2).mean())

def update_sol_hyd_val(i):
    ini_double = solution_double[i][0:3]
    param_double = solution_double[i][3:]
    Q_sim_double_val,[SM,UZ,LZ],[EA,R,RL,Q0,Q1] = HBV_sim(P_val,E_val,param_double,Case,ini_double,area)
    RMSE_val = RMSE_double_val[i]
    fig_8.title = 'RMSE = '+str("%.2f" % RMSE_val)
    return Q_sim_double_val, RMSE_val, i

def solution_selected_val(change):
    if pareto_front_val.selected == None:
        pareto_front_val.selected = [0]
    y_vals = update_sol_hyd_val(pareto_front_val.selected[0])[0]
    sim_hyd_double_val.y = y_vals

x_sc_pf_val = LinearScale()
y_sc_pf_val = LinearScale()

x_ax_pf_val = Axis(label='RMSE Low flows', scale=x_sc_pf_val)
y_ax_pf_val = Axis(label='RMSE High flows', scale=y_sc_pf_val, orientation='vertical')
  
fig_pf_val = plt.figure(title = 'Interactive Pareto front (Validation)', axes=[x_ax_pf_val, y_ax_pf_val],
                        layout={'width': '800px', 'height': '600px'}, animation_duration=1000)
pareto_front_selected = plt.scatter([results_double_low_val[pareto_front.selected[0]]],
                                    [results_double_high_val[pareto_front.selected[0]]],
                                    marker = 'cross',scales={'x': x_sc_pf_val, 'y': y_sc_pf_val},
                           colors=['black'], interactions={'hover':'tooltip','click': 'select'})
pareto_front_val = plt.scatter(results_double_low_val[:],results_double_high_val[:],
                               scales={'x': x_sc_pf_val, 'y': y_sc_pf_val}, colors=['deepskyblue'], 
                               interactions={'hover':'tooltip','click': 'select'})
pareto_front_val.unselected_style={'opacity': 0.4}
pareto_front_val.selected_style={'fill': 'red', 'stroke': 'yellow', 'width': '1125px', 'height': '125px'}
def_tt_val = Tooltip(fields=['x', 'y'],labels=['RMSE (High flows)', 'RMSE (Low flows)'], formats=['.1f', '.1f'])
pareto_front_val.tooltip=def_tt_val
pareto_front_val.selected = [pareto_front.selected[0]]

pareto_front_val.observe(solution_selected_val,'selected')    

x_sc_8 = DateScale()
y_sc_8 = LinearScale(max=1000)
    
x_ax_8 = Axis(label='date', scale=x_sc_8)
y_ax_8 = Axis(label='ML/day', scale=y_sc_8, orientation='vertical')

fig_8 = plt.figure(axes=[x_ax_8, y_ax_8], layout={'min_width': '1000px', 'max_height': '300px'}, animation_duration=1000,
                   scales={'x': x_sc_8, 'y': y_sc_8})
obs_hyd_val = plt.plot(Q_obs_date_val,Q_obs_val,colors=['black'])
sim_hyd_double_val = plt.plot(Q_obs_date_val,update_sol_hyd_val(pareto_front_val.selected[0])[0])
sim_hyd_double_val.observe(solution_selected_val, ['x', 'y'])
plt.VBox([fig_pf_val,fig_8])

We can see that the previously selected calibration for 2000 (represented by a cross) does not produce the same results when applied to 2001. The RMSE may be lower than before but do you see a good fit between the observed and the simulated hydrographs? Can you find a different point in the evaluation Pareto front that produces a "sensible compromise" between the objectives?

### References 

Bergström, S. (1992) The HBV model - its structure and applications. SMHI Reports RH, No. 4, Norrköping.

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

Yapo, P. O. et al (1998) Multi‐objective global optimization for hydrologic models, Journal of Hydrology, 204, 83–97.

#### Let's go to the next section!: [4.	Reservoir operation optimisation](4.%20Reservoir%20operation%20optimisation.ipynb)